In [1]:
import datetime
import pystac
import rioxarray

from geopandas import GeoSeries
from rasterio.features import shapes
from shapely.geometry import shape, mapping, MultiPolygon
from pystac import Catalog, Collection, Item, Asset

# Create STAC catalog from retiled images

In this notebook we illustrate how to create a STAC catalog for organizing the GeoTIFFs that will be produced during the retiling of the satellite imagery (either original images or mosaics). STAC catalogs entail the following elements: Catalog > Collection > Item (definitions given [here](https://stacspec.org/core.html)). In order to construct the catalog we use [PySTAC](https://pystac.readthedocs.io), which provides a representation of these elements as Python objects. For a PySTAC tutorial on how to construct a general-purpose catalog, have a look at this [tutorial](https://pystac.readthedocs.io/en/latest/tutorials/how-to-create-stac-catalogs.html).

In [2]:
# use composite image as test tile
asset_path = "TestDataSet/S2_comp_first.tif"

# define id's for the STAC objects
catalog_id = "ice-shelfs-antartica_retiled"
collection_id = "sentinel-2-L2A_retiled"
item_id = "PIG"
asset_id = "B2-B3-B4"

We use [rioxarray](https://corteva.github.io/rioxarray) to open the GeoTIFF:

In [3]:
# load image to extract information
bands = rioxarray.open_rasterio(asset_path)
bands

<xarray.DataArray (band: 3, y: 10002, x: 10004)>
[300180024 values with dtype=uint16]
Coordinates:
  * band         (band) int64 1 2 3
  * y            (y) float64 -2.667e+05 -2.667e+05 ... -3.667e+05 -3.667e+05
  * x            (x) float64 -1.654e+06 -1.654e+06 ... -1.554e+06 -1.554e+06
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     ('B4_first', 'B3_first', 'B2_first')
    grid_mapping:  spatial_ref

We start by creating an `Asset` object (see [docs](https://pystac.readthedocs.io/en/latest/api.html#asset)) associated with the tile (TBD: one can also add information about the bands included in the asset through the [EO extension](https://pystac.readthedocs.io/en/latest/api.html#eo-extension)):

In [4]:
# create Asset object
asset = Asset(
    href=asset_path,  # link to asset
    title=", ".join(bands.attrs['long_name']),
    media_type=pystac.MediaType.GEOTIFF # or COG - verify e.g. with with https://github.com/rouault/cog_validator 
)
asset

<Asset href=TestDataSet/S2_comp_first.tif>

We move on to create an `Item` object (see [docs](https://pystac.readthedocs.io/en/latest/api.html#item)), which will entail the asset we have previously defined (and potentially other assets). A STAC Items requires a bounding box and/or a geometry object, which we extract from the GeoTIFF. For the latter, we use `rasterio`'s `shapes` method to convert raster to vector information:  

In [5]:
# get tile bounds in WGS84, which is the standard in the GeoJSON format
bbox = bands.rio.transform_bounds("WGS84")
bbox

(-103.27724080817697,
 -75.56335580992435,
 -99.15858939334463,
 -74.49970054865628)

In [6]:
# determine footprint geometry
mask = bands.isnull()

polygons_and_values = shapes(
    mask.astype("int16").values,  # satisfy rasterio type requirements 
    transform=mask.rio.transform()
)
polygons = (shape(pol) for pol, val in polygons_and_values if val == 0.)
polygons = GeoSeries(polygons, crs=mask.spatial_ref.crs_wkt)
polygons = polygons.to_crs("WGS84")

geometry = mapping(polygons) if len(polygons)>1 else mapping(polygons[0])
geometry

{'type': 'Polygon',
 'coordinates': (((-99.15858939334463, -74.66974118070067),
   (-102.50015729056776, -74.49970054865628),
   (-103.27724080817697, -75.38244718914116),
   (-99.73727015169979, -75.56335580992435),
   (-99.15858939334463, -74.66974118070067)),)}

A datetime object (or a start/end datetime pair) is also required:

In [7]:
# for a mosaic, use earliest/latest datetimes of original images
start_datetime = datetime.datetime.fromisoformat("2019-11-01")
end_datetime = datetime.datetime.fromisoformat("2020-03-01") 

In [8]:
# create Item object
item = Item(
    id=item_id,
    geometry=geometry,
    bbox=bbox,
    datetime=None,  
    properties=dict(
        start_datetime=start_datetime.isoformat(),
        end_datetime=end_datetime.isoformat()   
    )
)

item.add_asset(asset_id, asset)
item

<Item id=PIG>

We then create a `Collection` object (docs [here](https://pystac.readthedocs.io/en/latest/api.html#collection)), which entails items that might be sharing some properties. Spatial and temporal extent of the collection need to be provided:  

In [9]:
# for multiple items, determine overall extents
items = (item, )

# spatial extent
footprints = (shape(i.geometry).envelope for i in items)
collection_bbox = MultiPolygon(footprints).bounds
spatial_extent = pystac.SpatialExtent(bboxes=[collection_bbox])

# temporal extent
start = (i.properties.get('start_datetime', i.datetime) for i in items)
start = sorted(start)[0]
end = (i.properties.get('end_datetime', i.datetime) for i in items)
end = sorted(end)[-1]
temporal_extent = pystac.TemporalExtent(
    intervals=[[datetime.datetime.fromisoformat(start), 
                datetime.datetime.fromisoformat(end)]]
)

extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)

In [10]:
# create Collection object
collection = Collection(
    id=collection_id,
    description="Retiled Sentinel-2-L2A Images",
    extent=extent,
    license="CC-BY-SA-4.0"
)

collection.add_items(items)
collection

<Collection id=sentinel-2-L2A_retiled>

Finally, we add the collection to a `Catalog` object ([here](https://pystac.readthedocs.io/en/latest/api.html#catalog) are the docs), which represents the "root" of the structure:

In [11]:
# create Catalog object
catalog = Catalog(
    id=catalog_id,
    description="Retiled Satellite Images for Antartica Ice Shelfs"
)

catalog.add_child(collection)
catalog

<Catalog id=ice-shelfs-antartica_retiled>

In [12]:
catalog.describe()

* <Catalog id=ice-shelfs-antartica_retiled>
    * <Collection id=sentinel-2-L2A_retiled>
      * <Item id=PIG>


We save the catalog as a self-contained object, i.e. using relative links within its elements. In this way, one can move the catalog's path or share it with other users, it will always possible to read it:  

In [13]:
catalog.normalize_and_save(f'./{catalog_id}', 'SELF_CONTAINED')